In [1]:
%pip install h2o

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 177.0 MB 131.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 91.5 MB/s eta 0:00:01
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=0533030546660b1c148809ab4577dcff647b0b881df5f2d4b3bf02c973cf12d2
  Stored in directory: /home/mk7516/.cache/pip/wheels/bf/d7/e0/905f059465fafaf07c9e79ba4c8d642ac646b01ea8c38b07d5
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=4f831c65e1c94dc6bcb6bd66006a00f5a1ba67ae0b30925cbe353bfd64aac98a
  Stored in directory: /home/mk7516/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built h2o future
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mk7516/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-18894125/tmp0fdvtmun
  JVM stdout: /state/partition1/job-18894125/tmp0fdvtmun/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-18894125/tmp0fdvtmun/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_mk7516_mpzdza
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup('australian')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(517, 15)

In [8]:
x = train.columns
y = "class"
x.remove(y)

In [9]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [10]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_1_20220501_164901_model_254


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,34.0,34.0,13814.0,5.0,11.0,8.735294,15.0,31.0,27.558823




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.03426676399475834
RMSE: 0.18511284124759778
LogLoss: 0.14718404878588576
Mean Per-Class Error: 0.03290185819953094
AUC: 0.9957829695110951
AUCPR: 0.9952841644584903
Gini: 0.9915659390221903

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.48106214224300337: 


,,0,1,Error,Rate
0,0,267.0,9.0,0.0326,(9.0/276.0)
1,1,8.0,233.0,0.0332,(8.0/241.0)
2,Total,275.0,242.0,0.0329,(17.0/517.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.885810,0.024515,0.903846,0.903846,0.844660,0.883495,0.893204
1,auc,0.930497,0.011425,0.930741,0.945616,0.913410,0.931395,0.931321
2,err,0.114190,0.024515,0.096154,0.096154,0.155340,0.116505,0.106796
3,err_count,11.800000,2.489980,10.000000,10.000000,16.000000,12.000000,11.000000
4,f0point5,0.868533,0.047063,0.883459,0.923695,0.797665,0.852018,0.885827
5,f1,0.879453,0.028775,0.903846,0.901961,0.836735,0.863636,0.891089
6,f2,0.891648,0.020340,0.925197,0.881226,0.879828,0.875576,0.896414
7,lift_top_group,2.157300,0.178511,2.080000,1.962264,2.288889,2.395349,2.060000
8,logloss,0.338110,0.034447,0.317048,0.313996,0.397707,0.336627,0.325171
9,max_per_class_error,0.139695,0.038430,0.129630,0.132075,0.206897,0.116667,0.113208



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-05-01 17:44:27,2 min 26.898 sec,0.0,0.498853,0.690854,0.500000,0.466151,1.000000,0.533849
1,,2022-05-01 17:44:27,2 min 26.912 sec,5.0,0.370182,0.457137,0.970939,0.968433,2.145228,0.096712
2,,2022-05-01 17:44:27,2 min 26.928 sec,10.0,0.305048,0.345449,0.977178,0.975679,2.145228,0.085106
3,,2022-05-01 17:44:27,2 min 26.944 sec,15.0,0.266884,0.277220,0.982042,0.980965,2.145228,0.073501
4,,2022-05-01 17:44:27,2 min 26.959 sec,20.0,0.240357,0.230290,0.986206,0.985559,2.145228,0.063830
5,,2022-05-01 17:44:27,2 min 26.975 sec,25.0,0.221730,0.197665,0.989266,0.988614,2.145228,0.054159
6,,2022-05-01 17:44:27,2 min 26.989 sec,30.0,0.200737,0.167450,0.993445,0.992895,2.145228,0.044487
7,,2022-05-01 17:44:28,2 min 27.002 sec,34.0,0.185113,0.147184,0.995783,0.995284,2.145228,0.032882



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,A8,356.853973,1.000000,0.623283
1,A2,51.594093,0.144580,0.090114
2,A3,31.651293,0.088695,0.055282
3,A13,27.685476,0.077582,0.048356
4,A9,26.533327,0.074353,0.046343
5,A5,17.065571,0.047822,0.029807
6,A14,14.827820,0.041552,0.025898
7,A7,12.619286,0.035363,0.022041
8,A10,9.687626,0.027147,0.016920
9,A4,6.043670,0.016936,0.010556


In [11]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_1_20220501_164901_model_254,0.934008,0.323031,0.927734,0.111913,0.308329,0.0950671
GBM_grid_1_AutoML_1_20220501_164901_model_122,0.931325,0.329412,0.92479,0.121527,0.310411,0.096355
GBM_grid_1_AutoML_1_20220501_164901_model_253,0.930408,0.33113,0.919935,0.12509,0.312705,0.0977845
GBM_grid_1_AutoML_1_20220501_164901_model_159,0.929724,0.329929,0.932932,0.11504,0.309697,0.0959122
GBM_grid_1_AutoML_1_20220501_164901_model_139,0.929641,0.330008,0.928817,0.120971,0.312358,0.0975675
GBM_grid_1_AutoML_1_20220501_164901_model_226,0.929423,0.332164,0.92284,0.125879,0.312838,0.0978674
GBM_grid_1_AutoML_1_20220501_164901_model_161,0.928589,0.330559,0.92668,0.12509,0.311035,0.0967426
GBM_lr_annealing_selection_AutoML_1_20220501_164901_select_model,0.928559,0.327285,0.920645,0.120708,0.309844,0.0960031
GBM_grid_1_AutoML_1_20220501_164901_model_9,0.928431,0.329689,0.920208,0.122519,0.311088,0.0967758
GBM_grid_1_AutoML_1_20220501_164901_model_23,0.928333,0.329443,0.917112,0.122489,0.30895,0.0954503
